In [2]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 14.3 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ln -s /content/drive/MyDrive/colab/didimdol /content/

In [4]:
!unzip /content/didimdol/dataset/iot_sensor_pickle_current.zip -d /content/dataset > null

In [1]:
import pandas as pd
import pickle
from didimdol.utils.currentdataset import CurrentDataset
# from model.RNNModel import RNNModel

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader 
import time
from tqdm import tqdm
import logging
import random
import os
import torchmetrics
from torch.utils.data.dataset import random_split

In [22]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    # # 기본변수, layer를 초기화해주는 생성자
    def __init__(self):
        super(LSTMModel, self).__init__()
        # self.hidden_dim = hidden_dim
        # self.seq_len = seq_len
        # self.output_dim = output_dim
        # self.layers = layers
        

        self.conv = nn.Conv1d(3, 16, 3, padding='same')
        self.maxpooling = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(16, 32, 3, padding='same')
        self.maxpooling2 = nn.MaxPool1d(2)

        # self.conv3 = nn.Conv1d(16, 32, 3, padding='same')
        # self.maxpooling3 = nn.MaxPool1d(2)

        # torch.Size([3, 1, 250])
        

        self.lstm = nn.LSTM(32, 64, num_layers=2,
                            # dropout = 0.1,
                            batch_first=True)
        self.fc = nn.Linear(64, 5, bias = True) 
        
    # 학습 초기화를 위한 함수
    # def reset_hidden_state(self): 
    #     self.hidden = (
    #             torch.zeros(self.layers, self.seq_len, self.hidden_dim),
    #             torch.zeros(self.layers, self.seq_len, self.hidden_dim))
    
    # 예측을 위한 함수
    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv(x)
        x = self.maxpooling(x)
        x = self.conv2(x)
        x = self.maxpooling2(x)
        # x = self.conv3(x)
        # x = self.maxpooling3(x)
        x = x.transpose(1, 2)
        x, _status = self.lstm(x)
        x = self.fc(x[:, -1])
        return x

In [23]:
dataset = CurrentDataset('didimdol/current_info.csv')

144730it [00:01, 127565.82it/s]


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
model = LSTMModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [26]:
train_loader = DataLoader(dataset,
                            batch_size=512,
                            shuffle=True,
                            num_workers=7,
                            pin_memory=True,
                            drop_last=False)

f1socre = torchmetrics.F1Score(num_classes = 5)

In [27]:
for epoch in range(100):
    progress = tqdm(train_loader)
    avg_cost = 0

    preds_hist = torch.tensor([],dtype= torch.int16).to(device)
    targets_hist = torch.tensor([],dtype= torch.int16).to(device)

    for data, targets in progress:
        data = data.to(device)
        targets = targets.to(device)

        preds = model(data)
        loss = criterion(preds, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_cost += loss
        progress.set_postfix({'loss' : loss.item()})

        preds = torch.max(preds.data, 1)[1]
        targets = torch.max(targets.data, 1)[1]

        preds_hist = torch.cat([preds_hist, preds])
        targets_hist = torch.cat([targets_hist, targets])

    f1 = f1socre(preds.to('cpu'), targets.to('cpu'))
    print('loss : {:.4f}, f1 : {:.4f}'.format(avg_cost / len(train_loader), f1))


100%|██████████| 283/283 [00:18<00:00, 15.39it/s, loss=0.924]


loss : 1.0838, f1 : 0.6098


100%|██████████| 283/283 [00:18<00:00, 15.21it/s, loss=0.959]


loss : 0.9637, f1 : 0.6590


100%|██████████| 283/283 [00:18<00:00, 15.56it/s, loss=1.02]


...

loss : 0.0779, f1 : 0.9827


100%|██████████| 283/283 [00:18<00:00, 15.36it/s, loss=0.0876]


loss : 0.0698, f1 : 0.9566


100%|██████████| 283/283 [00:18<00:00, 15.42it/s, loss=0.0393]

loss : 0.0685, f1 : 0.9827
